## File 02 - Feature Creation, test/train split, PCA
##### Group 12:

##### Hannah Schmuckler, mmc4cv

##### Rob Schwartz, res7cd

In this file, we create new features from our interaction-level dataset, handle obvious errors, and perform PCA. We will also create a downsampled dataset for comparison. We will output 4 files:

- `processed_data/engineered_features.parquet` # This is just a raw file containing all engineered feature rows
- `processed_data/train.parquet` #Our final training set
- `processed_data/test.parquet` #Our final test set
- `processed_data/ds_train.parquet` #Our downsampled training set - to see if downsampling might help performance. 
- `processed_data/ds_test.parquet` #Our test set - with PCA from the downsampled PCA. 


### Set up Spark session and data schema

We can specify more options in the SparkSession creator, but currently the options are at the default settings.

In [1]:
%%time
from pyspark.sql import SparkSession
from pyspark.sql import types as T
import matplotlib.pyplot as plt
from pyspark.sql.functions import col
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import *
import datetime as dt
from pyspark.sql.functions import translate

from pyspark.ml.feature import PCA as PCAml
from pyspark.ml.linalg import Vectors 
              
import copy
    
import sys
spark = SparkSession.builder \
        .appName("project") \
        .getOrCreate()

sc = spark.sparkContext

#schema = "`event_time` TIMESTAMP,`event_type` STRING,`product_id` INT,`category_id` BIGINT,`category_code` STRING,`brand` STRING,`price` FLOAT,`user_id` INT,`user_session` STRING"
#ddl_schema = T._parse_datatype_string(schema)

CPU times: user 632 ms, sys: 370 ms, total: 1 s
Wall time: 6.74 s


See https://docs.google.com/document/d/1NG4KGticBXn0D3PL5_zMxLV2Pr7A8PQtLcasxCOd1nA/edit for table of features.

### Read in data

In [2]:
%%time
full = spark.read.parquet("./processed_data/preprocessed_01.parquet")
m1 = spark.read.parquet("./processed_data/month_01_filtered.parquet") # This brings in the data we can create additional features from

CPU times: user 2.98 ms, sys: 1.68 ms, total: 4.66 ms
Wall time: 3.6 s


In [3]:
#print(full.count()) # 359105
#full.show(5)

In [4]:
#print(m1.count()) #15923973
#m1.show(5)

## Begin Creating Features
### Create each on an individual level, then join to full
##### NOTE: Must rename all features so that they do not contain parenthesis - not compatible with saving to parquet

_________________

#### Average Session Duration (avg_session_length)

In [5]:
session_ends = m1.groupBy('user_id', 'user_session').agg(max('event_time'), min('event_time'))

In [6]:
#session_ends.show(5)

In [7]:
session_ends = session_ends.withColumn('session_length', (col("max(event_time)").cast('long') - col("min(event_time)").cast('long')))

In [8]:
#session_ends.orderBy(col("session_length").desc()).show(5)
# NOTE: Lots of these sessions are unreasonably long

In [9]:
avg_sess = session_ends.groupBy('user_id').avg('session_length').withColumnRenamed('avg(session_length)', "avg_session_length")

In [10]:
#avg_sess.show(5)

In [11]:
full = full.join(avg_sess, full.user_id == avg_sess.user_id).drop(avg_sess.user_id)
#print(full.count()) # 359105
#full.show(5)

#### Std Deviation of session duration by person (sd_session_length)

In [12]:
#session_ends.show(5)

In [13]:
sd_session_length = session_ends.groupBy('user_id') \
                                 .agg(stddev('session_length')) \
                                 .withColumnRenamed("stddev_samp(session_length)", 'sd_session_length')

In [14]:
#sd_session_length.show(5)

In [15]:
full = full.join(sd_session_length, full.user_id == sd_session_length.user_id).drop(sd_session_length.user_id)

#### Average number of interactions per session (avg_interactions_per_session)

In [16]:
interactions_per_session = m1.groupBy('user_id', 'user_session').agg(count('event_type'))

In [17]:
#interactions_per_session.show(5)

In [18]:
avg_interactions_per_session = interactions_per_session.groupBy('user_id').avg('count(event_type)')

In [19]:
avg_interactions_per_session = avg_interactions_per_session.withColumnRenamed('avg(count(event_type))', "avg_interactions_per_session")

In [20]:
full = full.join(avg_interactions_per_session, full.user_id == avg_interactions_per_session.user_id).drop(avg_interactions_per_session.user_id)
#full.show(5)

#### Std Deviation of number of interactions per session per person (sd_interactions_per_session)

In [21]:
std_interactions_per_session = interactions_per_session.groupBy('user_id') \
                                                       .agg(stddev('count(event_type)')) \
                                                       .withColumnRenamed("stddev_samp(count(event_type))", 'sd_interactions_per_session')
#std_interactions_per_session.show(5)

In [22]:
full = full.join(std_interactions_per_session, full.user_id == std_interactions_per_session.user_id).drop(std_interactions_per_session.user_id)
#full.show(5)

#### Max number of interactions within one session (max_interactions_per_session)

In [23]:
max_interactions_per_session = interactions_per_session.groupBy('user_id').max('count(event_type)')

In [24]:
max_interactions_per_session = max_interactions_per_session.withColumnRenamed('max(count(event_type))', "max_interactions_per_session")

In [25]:
#max_interactions_per_session.show(1)

In [26]:
full = full.join(max_interactions_per_session, full.user_id == max_interactions_per_session.user_id).drop(max_interactions_per_session.user_id)
#full.show(5)

#### Percent of total events that are x (Purchase, Cart, View) ('purchase_pct_of_total_events', 'cart_pct_of_total_events', 'view_pct_of_total_events')

In [27]:
event_counts = m1.groupBy('user_id', 'user_session').pivot('event_type').agg(count('event_type'))
# Here the three types of event count are pivoted out for later tabulation

In [28]:
event_counts = event_counts.fillna(0) #replace nulls with 0 for math
#event_counts.show(5)

In [29]:
events_per_session = event_counts.withColumn('events_per_session_total', col('cart') + col('purchase') + col('view')) 
# Get total number of events per session

In [30]:
#events_per_session.show(5)

In [31]:
pct_events = events_per_session.groupBy('user_id').sum()

In [32]:
pct_totalevents = pct_events.withColumn('purchase_pct_of_total_events', col('sum(purchase)')/col('sum(events_per_session_total)')) \
                  .withColumn('view_pct_of_total_events', col('sum(view)')/col('sum(events_per_session_total)')) \
                  .withColumn('cart_pct_of_total_events', col('sum(cart)')/col('sum(events_per_session_total)'))

In [33]:
merge_me = pct_totalevents.select('user_id', 'purchase_pct_of_total_events', 'view_pct_of_total_events', 'cart_pct_of_total_events')

In [34]:
full = full.join(merge_me, full.user_id == merge_me.user_id).drop(merge_me.user_id)
#full.show(5)

#### Average number of purchases per session (avg_purchases_per_session)

In [35]:
avg_purchases_per_session = events_per_session.groupBy('user_id').avg('purchase').withColumnRenamed('avg(purchase)', "avg_purchases_per_session")

In [36]:
#avg_purchases_per_session.show(5)

In [37]:
full = full.join(avg_purchases_per_session, full.user_id == avg_purchases_per_session.user_id).drop(avg_purchases_per_session.user_id)
#full.show(5)

#### STD of number of purchases per session per person (sd_purchases_per_session)

In [38]:
std_purchases_per_session = events_per_session.groupBy('user_id') \
                                              .agg(stddev('purchase')) \
                                              .withColumnRenamed('stddev_samp(purchase)', "sd_purchases_per_session")
#std_purchases_per_session.show(5)

In [39]:
full = full.join(std_purchases_per_session, full.user_id == std_purchases_per_session.user_id).drop(std_purchases_per_session.user_id)
#full.show(5)

#### Total number of each type of event over whole month (monthlyCartTotal, monthlyPurchaseTotal, monthlyViewTotal)

In [40]:
event_counts_month = event_counts.groupBy('user_id').sum('cart', 'purchase', 'view')\
                     .withColumnRenamed('sum(cart)', 'cart_events') \
                     .withColumnRenamed('sum(purchase)', 'purchase_events') \
                     .withColumnRenamed('sum(view)', 'view_events')

In [41]:
#event_counts_month.show(5)

In [42]:
full = full.join(event_counts_month, full.user_id == event_counts_month.user_id).drop(event_counts_month.user_id)
#full.show(5)

#### Total number of sessions that contain event over whole month (sessions_with_purchase, sessions_with_cart, sessions_with_view)

In [43]:
events_over_month = events_per_session.withColumn('purchase_events', when(col('purchase') == 0, 0).otherwise(1)) \
                                      .withColumn('cart_events', when(col('cart')==0, 0).otherwise(1)) \
                                      .withColumn('view_events', when(col('view')==0, 0).otherwise(1))

In [44]:
num_sesh_containing_event = events_over_month.groupBy('user_id').sum('purchase_events', "cart_events", "view_events") \
                            .withColumnRenamed("sum(purchase_events)", "sessions_with_purchase") \
                            .withColumnRenamed("sum(cart_events)", "sessions_with_cart") \
                            .withColumnRenamed("sum(view_events)", "sessions_with_view")

In [45]:
#num_sesh_containing_event.show(5)

In [46]:
full = full.join(num_sesh_containing_event, full.user_id == num_sesh_containing_event.user_id).drop(num_sesh_containing_event.user_id)
#full.show(5)

#### Percent of individual's sessions that end in cart/purchase (pct_sessions_end_purchase, pct_sessions_end_cart)

In [47]:
session_ends2 = event_counts.withColumn('end_purchase', \
                                when(col('purchase') != 0, 1) \
                                .otherwise(0)) \
                            .withColumn('end_cart', \
                                when((col("purchase") == 0) & (col("cart") != 0), 1) \
                                .otherwise(0))
#session_ends2.show(5)

In [48]:
session_sum = session_ends2.groupBy('user_id').agg(count('user_session'), sum('end_purchase'), sum('end_cart'))
#session_sum.show(5)

In [49]:
session_sum = session_sum.withColumn('pct_sessions_end_purchase', col('sum(end_purchase)')/col('count(user_session)')) \
                         .withColumn('pct_sessions_end_cart', col('sum(end_cart)')/col('count(user_session)'))
#session_sum.show(5)

In [50]:
temp = session_sum.select('user_id', "pct_sessions_end_purchase", "pct_sessions_end_cart")
#temp.show(5)

+---------+-------------------------+---------------------+
|  user_id|pct_sessions_end_purchase|pct_sessions_end_cart|
+---------+-------------------------+---------------------+
|519121308|      0.16666666666666666|  0.16666666666666666|
|588954881|      0.09090909090909091|                  0.0|
|568888698|       0.3076923076923077|  0.23076923076923078|
|512700240|       0.5714285714285714|                  0.0|
|512975406|       0.1346153846153846|  0.23076923076923078|
+---------+-------------------------+---------------------+
only showing top 5 rows



In [51]:
full = full.join(temp, full.user_id == temp.user_id).drop(temp.user_id)
#full.show(5)

#### Add log of response column (T_total_spend_log) and total_spend_log, total_events_log, purchase_events_log, total_sessions_log

In [52]:
full = full \
          .withColumn("total_spend_log", log(col("total_spend")+0.001)) \
          .withColumn("total_events_log", log(col("total_events")+0.001)) \
          .withColumn("purchase_events_log", log(col("purchase_events")+0.001)) \
          .withColumn("total_sessions_log", log(col("total_sessions")+0.001)) \
          .withColumn("T_total_spend_log", log(col("T_total_spend")+0.001))

### Preview full dataframe

In [53]:
full.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- T_total_spend: double (nullable = true)
 |-- total_spend: double (nullable = true)
 |-- total_events: long (nullable = true)
 |-- total_sessions: long (nullable = true)
 |-- avg_session_length: double (nullable = true)
 |-- sd_session_length: double (nullable = true)
 |-- avg_interactions_per_session: double (nullable = true)
 |-- sd_interactions_per_session: double (nullable = true)
 |-- max_interactions_per_session: long (nullable = true)
 |-- purchase_pct_of_total_events: double (nullable = true)
 |-- view_pct_of_total_events: double (nullable = true)
 |-- cart_pct_of_total_events: double (nullable = true)
 |-- avg_purchases_per_session: double (nullable = true)
 |-- sd_purchases_per_session: double (nullable = true)
 |-- cart_events: long (nullable = true)
 |-- purchase_events: long (nullable = true)
 |-- view_events: long (nullable = true)
 |-- sessions_with_purchase: long (nullable = true)
 |-- sessions_with_cart: long (nullable =

In [54]:
full.show(1)

+---------+------------------+------------------+------------+--------------+------------------+------------------+----------------------------+---------------------------+----------------------------+----------------------------+------------------------+------------------------+-------------------------+------------------------+-----------+---------------+-----------+----------------------+------------------+------------------+-------------------------+---------------------+----------------+-----------------+-------------------+------------------+-----------------+
|  user_id|     T_total_spend|       total_spend|total_events|total_sessions|avg_session_length| sd_session_length|avg_interactions_per_session|sd_interactions_per_session|max_interactions_per_session|purchase_pct_of_total_events|view_pct_of_total_events|cart_pct_of_total_events|avg_purchases_per_session|sd_purchases_per_session|cart_events|purchase_events|view_events|sessions_with_purchase|sessions_with_cart|sessions_with_

In [55]:
#full.count() 359105

359105

#### Coerce NAs in standard deviation columnns to 0 (If there's only one, after all, the standard deviation IS 0!)

In [56]:
full = full.fillna(0, subset=['sd_session_length', 'sd_interactions_per_session', 'sd_purchases_per_session'])

### Remove errors/extreme outliers

In [57]:
# Look at abnormally large number of sessions
#full.select('total_sessions').sort(desc('total_sessions')).show(50)

In [58]:
# More than 10 sessions a day for a month (300) seems likely to be an error. Removing those rows. 
full = full.filter(col('total_sessions') <= 300)
#full.select('total_sessions').sort(desc('total_sessions')).show(5)
#full.count() #

359057

In [59]:
# Look at abnormally long sessions
# Average session length greater than 24 hours,or 86400 seconds, is almost certainly an error.
#full.select('avg_session_length').sort(desc('avg_session_length')).show(100)

In [60]:
# Remove individuals with average session length greater than 24 hours (86400 seconds)
full = full.filter(col('avg_session_length') <= 86400)
#full.select('avg_session_length').sort(desc('avg_session_length')).show(5)
#full.count() # 359057

351269

In [61]:
# Look at outliers by total spend. 
#full.select('total_spend', 'purchase_events', 'T_total_spend').sort(desc('total_spend')).show(50)
#full.select('total_spend').summary().show()

In [62]:
# Remove those that spend > 100k a month... they probably aren't our 'usual' customer. "Mega" customers are a different analysis. 
full = full.filter(col('total_spend') <= 100000)
#full.select('total_spend').sort(desc('total_spend')).show(5)
#full.count() #349358

349358

In [63]:
# Look at abnormally large numbers of events
# Greater than 3000 events is almost certainly a mistake. That's more than 100 events every day of the month
#full.select('total_events').sort(desc('total_events')).show(50)
#full.select('total_events').summary().show()

In [64]:
# Remove extreme> 3000 events a month
full = full.filter(col('total_events') <= 3000)
#full.select('total_events').sort(desc('total_events')).show(5)
#full.count() #348356

348356

#### Save as parquet.

In [65]:
%%time
full.write.mode("overwrite").parquet("./processed_data/engineered_features.parquet")

CPU times: user 9.46 ms, sys: 4.49 ms, total: 13.9 ms
Wall time: 1min 27s


In [66]:
%%time
train, test = full.randomSplit([.8, .2], seed=42)

CPU times: user 1.56 ms, sys: 1.1 ms, total: 2.65 ms
Wall time: 28.9 ms


#### After building our models using the original data, we became curious about how results would change if we downsampled the 0s in our response variable, since our data is zero-inflated. Below, we'll downsample the data. This new training set will be subbed in for the old, and original test set performance compared. (We have to do the downsampling here because otherwise we might wind up with overlap between it and the original test set.)

In [67]:
%%time
down_0 = train.filter(col('T_total_spend') == 0)
down_1 = train.filter(col('T_total_spend') != 0)

CPU times: user 361 µs, sys: 2.34 ms, total: 2.7 ms
Wall time: 41.7 ms


In [68]:
%%time
#down_0.count() #209502

CPU times: user 6.74 ms, sys: 4.23 ms, total: 11 ms
Wall time: 1min 5s


209502

In [69]:
%%time
#down_1.count() #69465

CPU times: user 6.07 ms, sys: 4.88 ms, total: 10.9 ms
Wall time: 1min 3s


69465

#### Looks like there's about 3x as many 0s as there are non-0s. Downsample to even that out. 

In [70]:
%%time
keep, _ = down_0.randomSplit([.33, .67], seed=42)

CPU times: user 0 ns, sys: 2.49 ms, total: 2.49 ms
Wall time: 14.8 ms


In [71]:
%%time
#keep.count() #69343

CPU times: user 5.64 ms, sys: 4.57 ms, total: 10.2 ms
Wall time: 1min 2s


69343

#### Join them back into one dataset

In [72]:
%%time
df_ds = down_1.union(keep)

CPU times: user 547 µs, sys: 379 µs, total: 926 µs
Wall time: 30.6 ms


#### Save as parquet

In [73]:
%%time
df_ds.write.mode("overwrite").parquet("./processed_data/ds_engineered_features.parquet")

CPU times: user 9.46 ms, sys: 4.67 ms, total: 14.1 ms
Wall time: 1min 28s


## Prepare PCA
#### Purchased items in month 1, converted to PCA (pca_purchases)

Note: Unlike all of the other preprocessing, we need to train the PCA model on the training set, then implement it on the test set. For this reason it comes after the train/test split.

In [74]:
%%time

# Create a function that prepares a dataset for PCA.

def pca_prepare_on_subset(subset_df, limited_columns=[]):
    # Only get this data from the training (or test) set
    m1_subset = m1.join(subset_df,'user_id','leftsemi')

    # Remove the periods from the dataframe category_code and replace with dashes. PySpark does not do well with periods in column
    #  names, for some reason
    m1_stripped = m1.withColumn('category_code_s', translate('category_code', '.', '-'))

    # Pivot so that each category of purchase becomes a column. It will have the number of purchases a given person made in that category. 
    cats = m1_stripped.filter(m1.event_type == "purchase").groupBy('user_id').pivot('category_code_s').count().na.fill(0)

    pca_input_cols = [cols for cols in cats.columns if cols!='user_id' and cols!='null']
        
    # Make a new copy of columns (this is from the training set to the test set, in order to filter out other columns)
    if(limited_columns==[]):
        limited_columns = copy.deepcopy(pca_input_cols)
        limited_columns.append('user_id')
    else:
        cats = cats.select(*limited_columns) # This is for the test set 
        # print(cats.schema)

    # Transform columns into a sparse vector (prepare for PCA)
    assembler = VectorAssembler(
        inputCols=pca_input_cols,
        outputCol="to_pca_columns")
    
    # Create sparse vector
    pca_df = assembler.transform(cats)
    return limited_columns, pca_df
    

CPU times: user 0 ns, sys: 5 µs, total: 5 µs
Wall time: 8.11 µs


### PCA on Original Data


In [75]:
# Get columns from training set and get training df
limited_columns, train_pre_pca = pca_prepare_on_subset(train)
# Limit to these columns on the test set and get test df
_, test_pre_pca = pca_prepare_on_subset(test, limited_columns=limited_columns)

# Visualize what this looks like
train_pre_pca.select(["user_id","to_pca_columns"]).show(2, truncate=False)

+---------+-----------------------+
|user_id  |to_pca_columns         |
+---------+-----------------------+
|512372691|(135,[0,112],[1.0,1.0])|
|512517137|(135,[85],[1.0])       |
+---------+-----------------------+
only showing top 2 rows



#### k = 10

In [76]:
%%time
# Create new PCA instance
pca = PCAml(k=10, inputCol="to_pca_columns", outputCol="pca_purchases10")
# Fit on training data
model = pca.fit(train_pre_pca)

# Transform training and test sets
train_with_pca = model.transform(train_pre_pca)
test_with_pca = model.transform(test_pre_pca)

# Merge PCA df back into full training set
join_train_df = train_with_pca.select(["user_id","pca_purchases10"])
train = train.join(join_train_df, train.user_id == join_train_df.user_id).drop(join_train_df.user_id)

# Merge PCA df back into full test set
join_test_df = test_with_pca.select(["user_id","pca_purchases10"])
test_o = test.join(join_test_df, test.user_id == join_test_df.user_id).drop(join_test_df.user_id)


CPU times: user 16.9 ms, sys: 7.45 ms, total: 24.3 ms
Wall time: 20 s


#### k = 20

In [77]:
%%time
# Create new PCA instance
pca = PCAml(k=20, inputCol="to_pca_columns", outputCol="pca_purchases20")
# Fit on training data
model = pca.fit(train_pre_pca)

# Transform training and test sets
train_with_pca = model.transform(train_pre_pca)
test_with_pca = model.transform(test_pre_pca)

# Merge PCA df back into full training set
join_train_df = train_with_pca.select(["user_id","pca_purchases20"])
train = train.join(join_train_df, train.user_id == join_train_df.user_id).drop(join_train_df.user_id)

# Merge PCA df back into full test set
join_test_df = test_with_pca.select(["user_id","pca_purchases20"])
test_o = test_o.join(join_test_df, test_o.user_id == join_test_df.user_id).drop(join_test_df.user_id)


CPU times: user 16.2 ms, sys: 6.05 ms, total: 22.3 ms
Wall time: 15.4 s


#### k = 50

In [78]:
%%time
# Create new PCA instance
pca = PCAml(k=50, inputCol="to_pca_columns", outputCol="pca_purchases50")
# Fit on training data
model = pca.fit(train_pre_pca)

# Transform training and test sets
train_with_pca = model.transform(train_pre_pca)
test_with_pca = model.transform(test_pre_pca)

# Merge PCA df back into full training set
join_train_df = train_with_pca.select(["user_id","pca_purchases50"])
train = train.join(join_train_df, train.user_id == join_train_df.user_id).drop(join_train_df.user_id)

# Merge PCA df back into full test set
join_test_df = test_with_pca.select(["user_id","pca_purchases50"])
test_o = test_o.join(join_test_df, test_o.user_id == join_test_df.user_id).drop(join_test_df.user_id)


CPU times: user 17.2 ms, sys: 6.13 ms, total: 23.3 ms
Wall time: 14.8 s


#### k = 100

In [79]:
%%time
# Create new PCA instance
pca = PCAml(k=100, inputCol="to_pca_columns", outputCol="pca_purchases100")
# Fit on training data
model = pca.fit(train_pre_pca)

# Transform training and test sets
train_with_pca = model.transform(train_pre_pca)
test_with_pca = model.transform(test_pre_pca)

# Merge PCA df back into full training set
join_train_df = train_with_pca.select(["user_id","pca_purchases100"])
train = train.join(join_train_df, train.user_id == join_train_df.user_id).drop(join_train_df.user_id)

# Merge PCA df back into full test set
join_test_df = test_with_pca.select(["user_id","pca_purchases100"])
test_o = test_o.join(join_test_df, test_o.user_id == join_test_df.user_id).drop(join_test_df.user_id)


CPU times: user 20.5 ms, sys: 2.18 ms, total: 22.7 ms
Wall time: 15.5 s


In [80]:
#train.show(5, truncate=False)
#test_o.show(5, truncate=False)

#### Write train and test

In [81]:
%%time
train.write.mode("overwrite").parquet("./processed_data/train.parquet")
test_o.write.mode("overwrite").parquet("./processed_data/test.parquet")

CPU times: user 12.9 ms, sys: 10.2 ms, total: 23.1 ms
Wall time: 2min 34s


### PCA on Downsampled Data


In [82]:
# Get columns from training set and get training df
limited_columns, train_pre_pca = pca_prepare_on_subset(df_ds)
# Limit to these columns on the test set and get test df
_, test_pre_pca = pca_prepare_on_subset(test, limited_columns=limited_columns)

# Visualize what this looks like
train_pre_pca.select(["user_id","to_pca_columns"]).show(2, truncate=False)

+---------+-----------------------+
|user_id  |to_pca_columns         |
+---------+-----------------------+
|512372691|(135,[0,112],[1.0,1.0])|
|512517137|(135,[85],[1.0])       |
+---------+-----------------------+
only showing top 2 rows



#### k = 10

In [83]:
%%time
# Create new PCA instance
pca = PCAml(k=10, inputCol="to_pca_columns", outputCol="pca_purchases10")
# Fit on training data
model = pca.fit(train_pre_pca)

# Transform training and test sets
train_with_pca = model.transform(train_pre_pca)
test_with_pca = model.transform(test_pre_pca)

# Merge PCA df back into full training set
join_train_df = train_with_pca.select(["user_id","pca_purchases10"])
train_ds = df_ds.join(join_train_df, df_ds.user_id == join_train_df.user_id).drop(join_train_df.user_id)

# Merge PCA df back into full test set
join_test_df = test_with_pca.select(["user_id","pca_purchases10"])
test_ds = test.join(join_test_df, test.user_id == join_test_df.user_id).drop(join_test_df.user_id)


CPU times: user 12.2 ms, sys: 7.47 ms, total: 19.6 ms
Wall time: 9.3 s


#### k = 20

In [84]:
%%time
# Create new PCA instance
pca = PCAml(k=20, inputCol="to_pca_columns", outputCol="pca_purchases20")
# Fit on training data
model = pca.fit(train_pre_pca)

# Transform training and test sets
train_with_pca = model.transform(train_pre_pca)
test_with_pca = model.transform(test_pre_pca)

# Merge PCA df back into full training set
join_train_df = train_with_pca.select(["user_id","pca_purchases20"])
train_ds = train_ds.join(join_train_df, train_ds.user_id == join_train_df.user_id).drop(join_train_df.user_id)

# Merge PCA df back into full test set
join_test_df = test_with_pca.select(["user_id","pca_purchases20"])
test_ds = test_ds.join(join_test_df, test_ds.user_id == join_test_df.user_id).drop(join_test_df.user_id)


CPU times: user 14.4 ms, sys: 5.3 ms, total: 19.7 ms
Wall time: 8.92 s


#### k = 50

In [85]:
%%time
# Create new PCA instance
pca = PCAml(k=50, inputCol="to_pca_columns", outputCol="pca_purchases50")
# Fit on training data
model = pca.fit(train_pre_pca)

# Transform training and test sets
train_with_pca = model.transform(train_pre_pca)
test_with_pca = model.transform(test_pre_pca)

# Merge PCA df back into full training set
join_train_df = train_with_pca.select(["user_id","pca_purchases50"])
train_ds = train_ds.join(join_train_df, train_ds.user_id == join_train_df.user_id).drop(join_train_df.user_id)

# Merge PCA df back into full test set
join_test_df = test_with_pca.select(["user_id","pca_purchases50"])
test_ds = test_ds.join(join_test_df, test_ds.user_id == join_test_df.user_id).drop(join_test_df.user_id)


CPU times: user 14.2 ms, sys: 4.5 ms, total: 18.7 ms
Wall time: 9.03 s


#### k = 100

In [86]:
%%time
# Create new PCA instance
pca = PCAml(k=100, inputCol="to_pca_columns", outputCol="pca_purchases100")
# Fit on training data
model = pca.fit(train_pre_pca)

# Transform training and test sets
train_with_pca = model.transform(train_pre_pca)
test_with_pca = model.transform(test_pre_pca)

# Merge PCA df back into full training set
join_train_df = train_with_pca.select(["user_id","pca_purchases100"])
train_ds = train_ds.join(join_train_df, train_ds.user_id == join_train_df.user_id).drop(join_train_df.user_id)

# Merge PCA df back into full test set
join_test_df = test_with_pca.select(["user_id","pca_purchases100"])
test_ds = test_ds.join(join_test_df, test_ds.user_id == join_test_df.user_id).drop(join_test_df.user_id)


CPU times: user 11.6 ms, sys: 7.63 ms, total: 19.2 ms
Wall time: 8.98 s


In [87]:
#train_ds.show(1, truncate=False)
#test_ds.show(1, truncate=False)

### Write downsampled train and test

In [88]:
%%time
train_ds.write.mode("overwrite").parquet("./processed_data/ds_train.parquet")
test_ds.write.mode("overwrite").parquet("./processed_data/ds_test.parquet")

CPU times: user 13.8 ms, sys: 9.65 ms, total: 23.5 ms
Wall time: 2min 31s
